In [1]:
import pandas as pd
import ast 
from scipy.spatial import distance

In [8]:
f = ("/home/vsl333/cultural_values/gpt_csv_data/cvqa_Pakistan_gpt_output.csv")
data = pd.read_csv(f)
print(data.shape)
data['country'].unique()
data.head(2)

(2110, 16)


,id,image_path,country,question_text,country_prompt,generic_prompt,option_labels,full_options,selection_answers,image_code,category,language,top_k,output_txt,top_k_p,top_k_pv
0,5865940254275595679_1,/home/vsl333/cultural_values/cvqa_chosen/image...,Pakistan,"In your view, how often do the following thing...",You are in a simulation game where you live i...,You are in a simulation game where you live i...,"['A', 'B', 'C', 'D', 'E', 'F', 'G']","(A) Very often, (B) Fairly often, (C) Not ofte...","[23.7, 27.7, 20.9, 20.9, 6.6, 0.4, 0.0]",cvqa,Vehicles and Transportation,Urdu,"[('A', -0.16150509), ('B', -1.9115051), ('C', ...","```json\n{'top_choice': 'A', 'second_choice': ...","[('A', 85.09), ('B', 14.79), ('C', 0.13), ('D'...","[85.09, 14.79, 0.13, 0.0, 0.0, 0.0, 0.0]"
1,5865940254278371030_1,/home/vsl333/cultural_values/cvqa_chosen/image...,Pakistan,I am going to name a number of organizations. ...,You are in a simulation game where you live i...,You are in a simulation game where you live i...,"['A', 'B', 'C', 'D', 'E', 'F', 'G']","(A) A great deal, (B) Quite a lot, (C) Not ver...","[19.6, 30.1, 14.4, 20.6, 14.2, 1.1, 0.0]",cvqa,Plants and animal,Urdu,"[('A', -1.9149379), ('B', -0.41493788), ('C', ...","```json\n{'top_choice': 'B', 'second_choice': ...","[('A', 14.74), ('B', 66.04), ('C', 18.92), ('D...","[14.74, 66.04, 18.92, 0.27, 0.04, 0.0, 0.0]"


In [9]:

# Example gt and pred (list of lists)
pak_gt = [ast.literal_eval(x) for x in data['selection_answers'].tolist()]
pak_pred = [ast.literal_eval(x) for x in data['top_k_pv'].tolist()]

# Original DataFrame, retaining all columns from `data`
df = data.copy()

# Keep track of matching indices
matching_indices = [i for i, (g, p) in enumerate(zip(pak_gt, pak_pred)) if len(g) == len(p)]

# Filter rows where len(g) == len(p)
filtered_gt_pred = [(g, p) for g, p in zip(pak_gt, pak_pred) if len(g) == len(p)]

# If there are any matching rows
if filtered_gt_pred:
    gt_filtered, pred_filtered = zip(*filtered_gt_pred)

    # Calculate JSD
    jsd = [distance.jensenshannon(g, p) for g, p in zip(gt_filtered, pred_filtered)]
    similarity = [1 - value for value in jsd]

    # Add JSD and similarity values back to the DataFrame
    df.loc[matching_indices, 'jsd'] = jsd
    df.loc[matching_indices, 'similarity'] = similarity
else:
    df['jsd'] = None  # or handle the case accordingly
    df['similarity'] = None

# Set JSD and similarity to NaN or a placeholder for rows that were filtered out
df['jsd'] = pd.to_numeric(df['jsd'], errors='coerce')
df['similarity'] = pd.to_numeric(df['similarity'], errors='coerce')

# Calculate mean of jsd and similarity per country
mean_per_country = df.groupby('country')[['jsd', 'similarity']].mean().reset_index()

/home/vsl333/cultural_values/culture-values/lib/python3.9/site-packages/scipy/spatial/distance.py:1262: RuntimeWarning: invalid value encountered in divide
  q = q / np.sum(q, axis=axis, keepdims=True)
/home/vsl333/cultural_values/culture-values/lib/python3.9/site-packages/scipy/spatial/distance.py:1261: RuntimeWarning: invalid value encountered in divide
  p = p / np.sum(p, axis=axis, keepdims=True)


In [10]:
mean_per_country

,country,jsd,similarity
0,Pakistan,0.498864,0.501136
